In [ ]:
!nvidia-smi

Sun Oct 24 12:24:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip -q install torchaudio
# !pip -q install datasets

import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
from shutil import copyfile

import librosa
import librosa.display
import scipy.io.wavfile as wav

import IPython.display as ipd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.6 MB/s eta 0:00:31tcmalloc: large alloc 1147494400 bytes == 0x564f5dfec000 @  0x7f4b57d48615 0x564f256904cc 0x564f2577047a 0x564f256932ed 0x564f25784e1d 0x564f25706e99 0x564f257019ee 0x564f25694bda 0x564f25706d00 0x564f257019ee 0x564f25694bda 0x564f25703737 0x564f25785c66 0x564f25702daf 0x564f25785c66 0x564f25702daf 0x564f25785c66 0x564f25702daf 0x564f25695039 0x564f256d8409 0x564f25693c52 0x564f25706c25 0x564f257019ee 0x564f25694bda 0x564f25703737 0x564f257019ee 0x564f25694bda 0x564f25702915 0x564f25694afa 0x564f25702c0d 0x564f257019ee
     |████████████████████████████████| 881.9 MB 20 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu111 requires torch==1.9.0, but you have torch 1.10.0 which is incompatible.
torchtext 0.10.0 re

In [ ]:
import pandas as pd
from torch.utils.data import Dataset

import torch

In [ ]:
train_rc = pd.read_csv('/content/drive/MyDrive/Tesi/train_rc.csv')
test_rc = pd.read_csv('/content/drive/MyDrive/Tesi/test_rc.csv')

In [ ]:
!7z x /content/drive/MyDrive/Tesi/dataset_bandPass2.zip -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Tesi/                                      1 file, 569072075 bytes (543 MiB)

Extracting archive: /content/drive/MyDrive/Tesi/dataset_bandPass2.zip
 59% 4096 Open              --
Path = /content/drive/MyDrive/Tesi/dataset_bandPass2.zip
Type = zip
Physical Size = 569072075

  0%      0% 21 - dataset_bandPass2/145_3b2_Ar_mc_AKGC417L_1_bandPass2.wav                                                                    0% 65 - dataset_bandPass2/163_2b2_Lr_mc_AKGC417L_3_bandPass2.wav

In [ ]:
print(train_rc.shape)
print(test_rc.shape)

(5518, 2)
(1380, 2)


In [ ]:
train_rc

,filename,label
0,194_1b1_Pr_sc_Meditron_7,none
1,113_1b1_Ar_sc_Litt3200_2,none
2,211_1p2_Pr_mc_AKGC417L_11,none
3,120_1b1_Lr_sc_Meditron_26,none
4,203_1p4_Pr_mc_AKGC417L_0,none
...,...,...
5513,158_1p4_Tc_mc_AKGC417L_8,none
5514,154_1b3_Ll_mc_AKGC417L_0,crackle
5515,113_1b1_Lr_sc_Litt3200_3,none
5516,204_2b5_Al_mc_AKGC417L_2,wheeze


In [ ]:
print(train_rc['label'].unique())

['none' 'wheeze' 'crackle' 'both']


In [ ]:
train_binary = train_rc.copy()
train_binary['label'] = (train_binary['label'] =='none')*1
train_binary['filename']="/content/dataset_bandPass2/"+train_binary['filename']+'_bandPass2.wav'
train_binary.head()

,filename,label
0,/content/dataset_bandPass2/194_1b1_Pr_sc_Medit...,1
1,/content/dataset_bandPass2/113_1b1_Ar_sc_Litt3...,1
2,/content/dataset_bandPass2/211_1p2_Pr_mc_AKGC4...,1
3,/content/dataset_bandPass2/120_1b1_Lr_sc_Medit...,1
4,/content/dataset_bandPass2/203_1p4_Pr_mc_AKGC4...,1


In [ ]:
test_binary = test_rc.copy()
test_binary['label'] = (test_binary['label'] =='none')*1
test_binary['filename']="/content/dataset_bandPass2/"+test_binary['filename']+'_bandPass2.wav'
test_binary.head()

,filename,label
0,/content/dataset_bandPass2/157_1b1_Pr_sc_Medit...,0
1,/content/dataset_bandPass2/130_2b2_Al_mc_AKGC4...,0
2,/content/dataset_bandPass2/210_1b1_Ar_sc_Medit...,1
3,/content/dataset_bandPass2/163_8b3_Ll_mc_AKGC4...,1
4,/content/dataset_bandPass2/178_1b6_Pl_mc_AKGC4...,1


In [ ]:
train_binary.groupby('label').count()

,filename
label,
0,2572
1,2946


In [ ]:
test_binary.groupby('label').count()

,filename
label,
0,684
1,696


In [ ]:
import torchaudio
duration = 4
freq = 16
class AudiosDataset(Dataset):
    def __init__(self, paths=None, targets=None, augment=False) -> None:
        self.paths = paths
        self.targets = targets
        self.augment = augment
        
    
    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int) -> dict:
        y, sr = torchaudio.load(self.paths.iloc[idx])#, normalization=True)
        """
        if self.augment:
            effects = [
                    ["lowpass", "-1", "300"], 
                    ["speed", f"{random.uniform(0.7, 1.3)}"],  # change speed
                  ]
            y, sr = torchaudio.sox_effects.apply_effects_tensor(
                y, sr, effects)
        #
            
        if self.augment:
            # augment sound in order to imitate the room change
            rir = self.rir[:, int(16000*1.1):int(16000*1.3)]
            rir = rir / torch.norm(rir, p=2)
            rir = torch.flip(rir, [1])
            y = torch.nn.functional.conv1d(y[None, ...], rir[None, ...])[0]
        
        """
        # here more augmentations could be used
        y = torchaudio.transforms.Resample(orig_freq=sr, new_freq=freq*1000)(y)

        """
        if self.augment:
            y = (0.5)*torch.randn(y.shape)
        """
        fixed_length = (freq*1000) * duration
        
        # returning result
        if y.shape[1] < fixed_length:
            y = torch.nn.functional.pad(
              y, (0, fixed_length - y.shape[1]))
        else:
            y = y[:, :fixed_length]
        # print(self.paths.iloc[idx])
        result = {"target":self.targets.iloc[idx], "representation":y}
        return result

In [ ]:
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    )
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
!pip install fairseq@git+https://github.com//pytorch/fairseq.git@f2146bdc7abf293186de9449bfa2272775e39e1d#egg=fairseq
!pip install sox

  Cloning https://github.com//pytorch/fairseq.git (to revision f2146bdc7abf293186de9449bfa2272775e39e1d) to /tmp/pip-install-_oyv6xpf/fairseq_39c042ec448e49568ed21dcca65ad36d
  Running command git clone -q https://github.com//pytorch/fairseq.git /tmp/pip-install-_oyv6xpf/fairseq_39c042ec448e49568ed21dcca65ad36d
  Running command git rev-parse -q --verify 'sha^f2146bdc7abf293186de9449bfa2272775e39e1d'
  Running command git fetch -q https://github.com//pytorch/fairseq.git f2146bdc7abf293186de9449bfa2272775e39e1d
  Running command git checkout -q f2146bdc7abf293186de9449bfa2272775e39e1d
  Running command git submodule update --init --recursive -q
  From https://github.com/ngoyal2707/Megatron-LM
   * branch            adb23324c222aad0aad89308e70302d996a5eaeb -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 123 kB 5.5 M

In [ ]:
from torch import nn
import torch.nn.functional as F

class ClassificationHead(nn.Module):
    """Head for mockingjay classification task."""

    def __init__(self):
        super().__init__()
        self.dense = nn.Linear(768, 512)#config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(0.5)
        self.out_proj = nn.Linear(512, 2)#config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class Classifier(nn.Module):
    def __init__(self, extractor):
        super().__init__()
        self.extractor = extractor
        self.classifier = ClassificationHead()
    def forward(self, x):
        features = self.extractor(torch.squeeze(x))
        features = features['last_hidden_state']
        res = torch.mean(features, dim=1)
        logit = self.classifier(res)
        return logit

In [ ]:
from tqdm import tqdm
import math
from torch import autograd
import datetime

def train(model, opt, scheduler, loss_fn, epochs, data_tr, data_val, lr, last_epoch=0, data_augmentation = False, max_stable=5):
    best_val_loss = 1e9
    counter = 0
    augmented = '_augmented' if data_augmentation else ''
    nan=False
    for epoch in range(last_epoch, epochs):
        correct_predictions = 0
        count=0
        #tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        avg_loss = 0
        model.train()  # train mode
        progress = tqdm(data_tr)
        for batch in progress:
            loss = 0
            # data to device
            X_batch, Y_batch = batch["representation"], batch["target"]
            # print(X_batch.shape)
            X_batch = X_batch.to(DEVICE)
            Y_batch = Y_batch.to(DEVICE)
            # set parameter gradients to zero
            opt.zero_grad()
            # forward
            Y_pred = model(X_batch)
            _, preds = torch.max(Y_pred, dim=1)
            preds = preds.to(DEVICE)
            Y_pred = Y_pred.to(DEVICE)

            loss = loss_fn(Y_pred, Y_batch)# forward-pass
            nan = math.isnan(loss)

            if nan:
              count=count+1
              nan=False
              continue
            correct_predictions += torch.sum(preds == Y_batch)
            progress.set_description(f'Loss: {loss:.2f} #nan->{count}')
            
            loss.backward()  # backward-pass
            opt.step()  # update weights
            if not scheduler is None:
                scheduler.step()
            # calculate loss to show the user
            avg_loss += loss
        avg_loss /= (len(data_tr)-(batch_size*count))
        avg_acc = correct_predictions.double() / ((len(data_tr)-count)*batch_size)
        
        print('loss: %f' % avg_loss)
        print('acc: %f' % avg_acc)
        # show intermediate results


        model.eval()  # testing mode
        val_loss = 0
        val_acc = 0
        print("start validation")
        count_loss_val=0
        progress_val = tqdm(data_val)
        correct_predictions_val = 0
        for v_b in progress_val:
            X_val, Y_val = v_b["representation"], v_b["target"]
            # X_val = X_val.to(DEVICE)#
            # Y_val = Y_val.to(DEVICE)#
            Y_hat = model(X_val.to(DEVICE)).detach().cpu()# detach and put into cpu
            
            _, preds_val = torch.max(Y_hat, dim=1)
            # preds_val = preds_val.to(DEVICE)#

            loss_val = loss_fn(Y_hat, Y_val)
            nan_loss = math.isnan(loss_val) #check loss==nan
            if nan_loss:
              count_loss_val = count_loss_val+1
              nan_loss = False
              continue
            #se trova un nan, salta tutto il batch
            correct_predictions_val += torch.sum(preds_val == Y_val)
            progress_val.set_description(f'Loss: {loss_val:.2f} #nan->{count_loss_val}')
            val_loss += loss_val

        val_loss /= (len(data_val)-(batch_size*count_loss_val))
        val_acc = correct_predictions_val.double() / ((len(data_val)-count_loss_val)*batch_size)
        print( f"validation loss: {val_loss}")
        print( f"validation accuracy: {val_acc}")
        if val_loss <= best_val_loss and val_loss > 0:
            counter = 0
            now = datetime.datetime.now()
            print(f'Save new model!- {now.day}/{now.month} h{now.hour+2}:{now.minute}')
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Tesi/Mockingjay/final_80-20_bandPass2_healthy-unhealthy_dropout=0.5_mockingjay_lr={lr}_batch={batch_size}_{freq}mhz_{duration}sec{augmented}.bin')
        else:
            counter += 1
        print("--------------------")
        if counter == max_stable:
            break

---------------------------

------------------------

In [ ]:
freq=16
duration=4
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Modello finale

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay')

Downloading: "https://github.com/s3prl/s3prl/archive/master.zip" to /root/.cache/torch/hub/master.zip
SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


[s3prl.upstream.experts] Warning: can not import s3prl.upstream.wav2vec2_hug.expert: No module named 'transformers'. Pass.
[s3prl.hub] Warning: can not import s3prl.upstream.wav2vec2_hug.hubconf: No module named 'transformers'. Please see upstream/wav2vec2_hug/README.md
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.enhancement_stft.expert: No module named 'asteroid'. Pass.
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.speech_translation.expert: No module named 'sentencepiece'. Pass.
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.speech_commands.expert: No module named 'catalyst'. Pass.
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.quesst14_dtw.expert: No module named 'dtw'. Pass.
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.separation_stft.expert: No module named 'asteroid'. Pass.
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
#DURAT=4SECONDI - final 80-20
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.79 #nan->0: 100%|██████████| 689/689 [14:47<00:00,  1.29s/it]


loss: 0.678127
acc: 0.579463
start validation


Loss: 0.60 #nan->0: 100%|██████████| 172/172 [01:29<00:00,  1.92it/s]


validation loss: 0.668724536895752
validation accuracy: 0.5973837209302325
Save new model!- 24/10 h14:48
--------------------
* Epoch 2/100


...

Loss: 0.48 #nan->0: 100%|██████████| 689/689 [14:44<00:00,  1.28s/it]


loss: 0.477277
acc: 0.778665
start validation


Loss: 1.19 #nan->0: 100%|██████████| 172/172 [01:30<00:00,  1.90it/s]


validation loss: 0.5783836841583252
validation accuracy: 0.7311046511627907
--------------------
* Epoch 6/100


Loss: 0.46 #nan->0:  80%|███████▉  | 551/689 [11:56<02:59,  1.30s/it]

--------------

---------------------

### Tentativi precedenti

In [ ]:
#DURAT=4SECONDI - 60-40
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.70 #nan->0: 100%|██████████| 517/517 [11:45<00:00,  1.37s/it]


loss: 0.720142
acc: 0.527321
start validation


Loss: 0.74 #nan->0: 100%|██████████| 344/344 [02:52<00:00,  2.00it/s]


validation loss: 0.7036807537078857
validation accuracy: 0.4262354651162791
Save new model!- 23/10 h21:49
--------------------
* Epoch 2/100


...

Loss: 0.66 #nan->0: 100%|██████████| 517/517 [11:49<00:00,  1.37s/it]


loss: 0.658599
acc: 0.617747
start validation


Loss: 0.32 #nan->0: 100%|██████████| 344/344 [02:53<00:00,  1.99it/s]


validation loss: 0.6780424118041992
validation accuracy: 0.5886627906976745
--------------------
* Epoch 9/100


Loss: 0.69 #nan->0:  80%|████████  | 415/517 [09:28<02:24,  1.41s/it]

In [ ]:
#DURAT=4SECONDI
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.60 #nan->0: 100%|██████████| 517/517 [11:45<00:00,  1.37s/it]


loss: 0.696844
acc: 0.555368
start validation


Loss: 0.29 #nan->0: 100%|██████████| 344/344 [02:49<00:00,  2.04it/s]


validation loss: 0.6887927055358887
validation accuracy: 0.5882994186046512
Save new model!- 23/10 h20:17
--------------------
* Epoch 2/100


...

validation loss: 1.0314395427703857
validation accuracy: 0.5363372093023255
--------------------
* Epoch 6/100


Loss: 0.72 #nan->0: 100%|██████████| 517/517 [11:46<00:00,  1.37s/it]


loss: 0.418252
acc: 0.813588
start validation


Loss: 0.05 #nan->0: 100%|██████████| 344/344 [02:50<00:00,  2.02it/s]


validation loss: 0.8849740028381348
validation accuracy: 0.6024709302325582
--------------------


----------------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay')

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master
SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


[s3prl.upstream.experts] Warning: can not import s3prl.upstream.wav2vec2_hug.expert: No module named 'transformers'. Pass.
[s3prl.hub] Warning: can not import s3prl.upstream.wav2vec2_hug.hubconf: No module named 'transformers'. Please see upstream/wav2vec2_hug/README.md
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.enhancement_stft.expert: No module named 'asteroid'. Pass.
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.speech_translation.expert: No module named 'sentencepiece'. Pass.
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.speech_commands.expert: No module named 'catalyst'. Pass.
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.quesst14_dtw.expert: No module named 'dtw'. Pass.
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.separation_stft.expert: No module named 'asteroid'. Pass.
Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9f

In [ ]:
model = Classifier(m)

In [ ]:
#DURAT=4SECONDI
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.54 #nan->0: 100%|██████████| 689/689 [14:56<00:00,  1.30s/it]


loss: 0.684792
acc: 0.569666
start validation


Loss: 0.66 #nan->0: 100%|██████████| 172/172 [01:23<00:00,  2.07it/s]


validation loss: 0.6332269906997681
validation accuracy: 0.6286337209302325
Save new model!- 21/10 h21:5
--------------------
* Epoch 2/100


...

Loss: 0.53 #nan->0: 100%|██████████| 689/689 [14:57<00:00,  1.30s/it]


loss: 0.479960
acc: 0.771771
start validation


Loss: 0.39 #nan->0: 100%|██████████| 172/172 [01:23<00:00,  2.06it/s]


validation loss: 0.5702335834503174
validation accuracy: 0.7107558139534884
--------------------
* Epoch 6/100


Loss: 0.27 #nan->0:  61%|██████▏   | 423/689 [09:11<05:54,  1.33s/it]

In [ ]:
#DURAT=4SECONDI
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Mockingjay/OK-bandPass2_healthy-unhealthy_dropout=0.5_mockingjay_lr=1e-05_batch=8_16mhz_4sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=4)
#4mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 5/100


Loss: 1.00 #nan->0: 100%|██████████| 689/689 [14:54<00:00,  1.30s/it]


loss: 0.470892
acc: 0.775762
start validation


Loss: 0.42 #nan->0: 100%|██████████| 172/172 [01:24<00:00,  2.03it/s]


validation loss: 0.5706985592842102
validation accuracy: 0.7332848837209303
Save new model!- 22/10 h11:15
--------------------
* Epoch 6/100


...

validation loss: 0.6916240453720093
validation accuracy: 0.7441860465116279
--------------------
* Epoch 10/100


Loss: 0.17 #nan->0: 100%|██████████| 689/689 [14:53<00:00,  1.30s/it]


loss: 0.272338
acc: 0.886974
start validation


Loss: 0.61 #nan->0: 100%|██████████| 172/172 [01:24<00:00,  2.03it/s]

validation loss: 0.7346189022064209
validation accuracy: 0.7420058139534884
--------------------


#### Training Mockingjay augmented

In [ ]:
df_rc = pd.read_csv('/content/drive/MyDrive/Tesi/dataset_binary_augmented.csv')

In [ ]:
#extract with 7zip and progressbar to VM Colab
!7z x /content/drive/MyDrive/Tesi/augmented_sample.zip -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Tesi/                                      1 file, 3408302740 bytes (3251 MiB)

Extracting archive: /content/drive/MyDrive/Tesi/augmented_sample.zip
 68% 4096 Open              --
Path = /content/drive/MyDrive/Tesi/augmented_sample.zip
Type = zip
Physical Size = 3408302740

  0%      0% 7 - augmented_sample/186_2b4_Tc_mc_AKGC417L_2_augmented.wav                                                                  0% 14 - augmented_sample/130_1p3_Ar_mc_AKGC417L_4_augmented.wav

In [ ]:
print(f'Numero entry dataframe: {len(df_rc)} - Numero file dataset: 12898')

Numero entry dataframe: 12898 - Numero file dataset: 12898


In [ ]:
df_rc.groupby('label').count()

,filename
label,
0,6256
1,6642


In [ ]:
save_path = '/content/drive/MyDrive/Tesi'
train_binary_augmented, test_binary_augmented = train_test_split(df_rc, test_size=0.2, random_state=42)#, stratify=df["label"])

train_binary_augmented = train_binary_augmented.reset_index(drop=True)
test_binary_augmented = test_binary_augmented.reset_index(drop=True)

train_binary_augmented.to_csv(f"{save_path}/train_binary_augmented.csv", sep="\t", index=False)
test_binary_augmented.to_csv(f"{save_path}/test_binary_augmented.csv", sep="\t", index=False)


print(train_binary_augmented.shape)
print(test_binary_augmented.shape)

(10318, 2)
(2580, 2)


-----------------------

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

freq=16
duration=5
train_ds = AudiosDataset(train_binary_augmented['filename'], train_binary_augmented['label'])
test_ds = AudiosDataset(test_binary_augmented['filename'], test_binary_augmented['label'])

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay')

Downloading: "https://github.com/s3prl/s3prl/archive/master.zip" to /root/.cache/torch/hub/master.zip


[s3prl.upstream.experts] Warning: can not import s3prl.upstream.wav2vec2_hug.expert: No module named 'transformers'. Pass.
[s3prl.hub] Warning: can not import s3prl.upstream.wav2vec2_hug.hubconf: No module named 'transformers'. Please see upstream/wav2vec2_hug/README.md


SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


[s3prl.downstream.experts] Warning: can not import s3prl.downstream.quesst14_dtw.expert: No module named 'dtw'. Pass.
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.speech_commands.expert: No module named 'catalyst'. Pass.
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.speech_translation.expert: No module named 'sentencepiece'. Pass.
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.enhancement_stft.expert: No module named 'asteroid'. Pass.
[s3prl.downstream.experts] Warning: can not import s3prl.downstream.separation_stft.expert: No module named 'asteroid'. Pass.
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =4e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, data_augmentation=True)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.64 #nan->0: 100%|██████████| 1289/1289 [36:21<00:00,  1.69s/it]


loss: 0.734494
acc: 0.523468
start validation


Loss: 0.69 #nan->0: 100%|██████████| 322/322 [03:08<00:00,  1.70it/s]


validation loss: 0.6846029758453369
validation accuracy: 0.5578416149068323
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.62 #nan->0: 100%|██████████| 322/322 [03:11<00:00,  1.68it/s]


validation loss: 0.6557091474533081
validation accuracy: 0.624611801242236
Save new model!
--------------------
* Epoch 9/100


Loss: 0.37 #nan->0: 100%|██████████| 1289/1289 [35:40<00:00,  1.66s/it]


loss: 0.653456
acc: 0.634019
start validation


Loss: 0.87 #nan->0:  57%|█████▋    | 185/322 [01:50<01:18,  1.74it/s]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Mockingjay/OK-healthy-unhealthy_mockingjay_lr=4e-06_batch=8_16mhz_5sec_augmented.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =4e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, data_augmentation=True)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert
# DALL'EPOCA 9

* Epoch 1/100


Loss: 0.81 #nan->0: 100%|██████████| 1289/1289 [35:57<00:00,  1.67s/it]


loss: 0.652957
acc: 0.634697
start validation


Loss: 0.59 #nan->0: 100%|██████████| 322/322 [03:13<00:00,  1.67it/s]


validation loss: 0.6494064927101135
validation accuracy: 0.6261645962732919
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.55 #nan->0: 100%|██████████| 1289/1289 [36:00<00:00,  1.68s/it]


loss: 0.597636
acc: 0.686191
start validation


Loss: 0.48 #nan->0: 100%|██████████| 322/322 [03:12<00:00,  1.67it/s]


validation loss: 0.6429112553596497
validation accuracy: 0.656444099378882
Save new model!
--------------------
* Epoch 6/100


Loss: 0.35 #nan->0:  54%|█████▍    | 702/1289 [19:34<16:34,  1.69s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Mockingjay/OK-healthy-unhealthy_mockingjay_lr=4e-06_batch=8_16mhz_5sec_augmented.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =4e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=13, data_augmentation=True)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert
# DALL'EPOCA 14

* Epoch 14/100


Loss: 0.45 #nan->0: 100%|██████████| 1289/1289 [35:04<00:00,  1.63s/it]


loss: 0.583941
acc: 0.700058
start validation


Loss: 0.52 #nan->0: 100%|██████████| 322/322 [03:06<00:00,  1.73it/s]


validation loss: 0.6484045386314392
validation accuracy: 0.6645962732919255
Save new model!
--------------------
* Epoch 15/100


Loss: 0.55 #nan->0: 100%|██████████| 1289/1289 [34:53<00:00,  1.62s/it]


loss: 0.569092
acc: 0.713247
start validation


Loss: 0.37 #nan->0: 100%|██████████| 322/322 [03:06<00:00,  1.72it/s]


validation loss: 0.6633588671684265
validation accuracy: 0.655667701863354
--------------------
* Epoch 16/100


Loss: 0.78 #nan->0:  81%|████████  | 1039/1289 [28:11<06:14,  1.50s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Mockingjay/OK-healthy-unhealthy_mockingjay_lr=4e-06_batch=8_16mhz_5sec_augmented.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =4e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=14, data_augmentation=True)
#16mhz 5secondi tanh dropout=0.7 batch=8 mockingjay


* Epoch 15/100


Loss: 0.69 #nan->0: 100%|██████████| 1289/1289 [33:40<00:00,  1.57s/it]


loss: 0.569813
acc: 0.715186
start validation


Loss: 0.37 #nan->0: 100%|██████████| 322/322 [03:00<00:00,  1.78it/s]


validation loss: 0.6614711880683899
validation accuracy: 0.6622670807453416
Save new model!
--------------------
* Epoch 16/100


...

Loss: 0.79 #nan->0: 100%|██████████| 1289/1289 [33:29<00:00,  1.56s/it]


loss: 0.492878
acc: 0.768037
start validation


Loss: 0.44 #nan->0: 100%|██████████| 322/322 [03:00<00:00,  1.78it/s]


validation loss: 0.6782204508781433
validation accuracy: 0.6793478260869565
--------------------
* Epoch 20/100


Loss: 0.36 #nan->0:  84%|████████▍ | 1082/1289 [28:05<05:25,  1.57s/it]

----------------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay')

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, data_augmentation=True)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.83 #nan->0: 100%|██████████| 1289/1289 [35:50<00:00,  1.67s/it]


loss: 0.722398
acc: 0.539178
start validation


Loss: 0.66 #nan->0: 100%|██████████| 322/322 [03:09<00:00,  1.70it/s]


validation loss: 0.684084951877594
validation accuracy: 0.5485248447204969
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.64 #nan->0: 100%|██████████| 322/322 [03:09<00:00,  1.70it/s]


validation loss: 0.6423956751823425
validation accuracy: 0.6424689440993789
Save new model!
--------------------
* Epoch 5/100


Loss: 0.31 #nan->0: 100%|██████████| 1289/1289 [35:03<00:00,  1.63s/it]


loss: 0.634503
acc: 0.660881
start validation


Loss: 0.60 #nan->0:   7%|▋         | 21/322 [00:15<02:57,  1.69it/s]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Mockingjay/OK-healthy-unhealthy_mockingjay_lr=1e-05_batch=8_16mhz_5sec_augmented.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=4, data_augmentation=True)
#16mhz 5secondi tanh dropout=0.7 batch=8 mockingjay


* Epoch 5/100


Loss: 0.56 #nan->0: 100%|██████████| 1289/1289 [34:35<00:00,  1.61s/it]


loss: 0.631308
acc: 0.653898
start validation


Loss: 0.55 #nan->0: 100%|██████████| 322/322 [03:04<00:00,  1.75it/s]


validation loss: 0.6375094652175903
validation accuracy: 0.6463509316770186
Save new model!
--------------------
* Epoch 6/100


...

validation loss: 0.6603034138679504
validation accuracy: 0.7088509316770186
--------------------
* Epoch 11/100


Loss: 0.25 #nan->0: 100%|██████████| 1289/1289 [34:17<00:00,  1.60s/it]


loss: 0.415220
acc: 0.813130
start validation


Loss: 0.47 #nan->0: 100%|██████████| 322/322 [03:03<00:00,  1.76it/s]


validation loss: 0.7420976161956787
validation accuracy: 0.6840062111801242
--------------------


-----------------------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay')

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =5e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, data_augmentation=True)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.64 #nan->0: 100%|██████████| 1289/1289 [34:29<00:00,  1.61s/it]


loss: 0.752656
acc: 0.508340
start validation


Loss: 0.63 #nan->0: 100%|██████████| 322/322 [03:00<00:00,  1.78it/s]


validation loss: 0.7003177404403687
validation accuracy: 0.5186335403726708
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.75 #nan->0: 100%|██████████| 1289/1289 [33:43<00:00,  1.57s/it]


loss: 0.717568
acc: 0.493697
start validation


Loss: 0.66 #nan->0: 100%|██████████| 322/322 [03:01<00:00,  1.78it/s]


validation loss: 0.6933491826057434
validation accuracy: 0.5186335403726708
--------------------
* Epoch 6/100


Loss: 0.70 #nan->0:  94%|█████████▍| 1212/1289 [31:11<01:56,  1.51s/it]

------------------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay')

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =5e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, data_augmentation=True)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 0.77 #nan->0: 100%|██████████| 1289/1289 [34:36<00:00,  1.61s/it]


loss: 0.725767
acc: 0.536462
start validation


Loss: 0.70 #nan->0: 100%|██████████| 322/322 [03:00<00:00,  1.78it/s]


validation loss: 0.6765249371528625
validation accuracy: 0.5920031055900621
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.54 #nan->0: 100%|██████████| 1289/1289 [33:37<00:00,  1.57s/it]


loss: 0.734026
acc: 0.503297
start validation


Loss: 0.74 #nan->0: 100%|██████████| 322/322 [03:00<00:00,  1.79it/s]


validation loss: 0.7006104588508606
validation accuracy: 0.4817546583850932
--------------------
* Epoch 5/100


Loss: 0.69 #nan->0:  12%|█▏        | 150/1289 [03:58<30:19,  1.60s/it]

--------------------------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay')

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, data_augmentation=True)
#16mhz 5secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.83 #nan->0: 100%|██████████| 1289/1289 [36:11<00:00,  1.68s/it]


loss: 0.728947
acc: 0.523662
start validation


Loss: 0.77 #nan->0: 100%|██████████| 322/322 [03:09<00:00,  1.70it/s]


validation loss: 0.6922784447669983
validation accuracy: 0.5232919254658385
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.65 #nan->0: 100%|██████████| 1289/1289 [35:23<00:00,  1.65s/it]


loss: 0.714563
acc: 0.540050
start validation


Loss: 0.75 #nan->0: 100%|██████████| 322/322 [03:11<00:00,  1.68it/s]


validation loss: 0.6817489266395569
validation accuracy: 0.5555124223602484
Save new model!
--------------------
* Epoch 9/100


Loss: 1.06 #nan->0:   7%|▋         | 86/1289 [02:26<32:49,  1.64s/it]

#### Training mockingjay not augmented

In [ ]:
# before running this part it is neccessary to run at least 1 batch through the model in order for inputs
# to adapt
# model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/AudioAlbert/3MHz-healthy-unhealthy_aalbert.bin'))

-----------------------

In [ ]:
freq=4
duration=6
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.72 #nan->0: 100%|██████████| 689/689 [05:58<00:00,  1.92it/s]


loss: 0.753183
acc: 0.511430
start validation


Loss: 0.67 #nan->0: 100%|██████████| 172/172 [00:27<00:00,  6.23it/s]


validation loss: 0.7618802785873413
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.6940693855285645
validation accuracy: 0.5174418604651163
--------------------
* Epoch 17/100


Loss: 0.66 #nan->0: 100%|██████████| 689/689 [04:58<00:00,  2.30it/s]


loss: 0.696164
acc: 0.519412
start validation


Loss: 0.68 #nan->0: 100%|██████████| 172/172 [00:27<00:00,  6.35it/s]


validation loss: 0.6929031014442444
validation accuracy: 0.5174418604651163
--------------------


-----------------------

In [ ]:
freq=4
duration=6
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.71 #nan->0: 100%|██████████| 689/689 [05:00<00:00,  2.29it/s]


loss: 0.726484
acc: 0.528483
start validation


Loss: 0.67 #nan->0: 100%|██████████| 172/172 [00:27<00:00,  6.28it/s]


validation loss: 0.6728041172027588
validation accuracy: 0.5559593023255814
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.87 #nan->0: 100%|██████████| 172/172 [00:27<00:00,  6.32it/s]


validation loss: 0.6146326661109924
validation accuracy: 0.7093023255813954
--------------------
* Epoch 9/100


Loss: 0.43 #nan->0: 100%|██████████| 689/689 [05:00<00:00,  2.29it/s]


loss: 0.411647
acc: 0.812954
start validation


Loss: 1.27 #nan->0:  52%|█████▏    | 89/172 [00:14<00:12,  6.74it/s]

-----------------


In [ ]:
freq=16
duration=6
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.85 #nan->0: 100%|██████████| 689/689 [18:30<00:00,  1.61s/it]


loss: 0.758175
acc: 0.500544
start validation


Loss: 0.69 #nan->0: 100%|██████████| 172/172 [01:35<00:00,  1.81it/s]


validation loss: 0.6927146911621094
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.6975802183151245
validation accuracy: 0.5174418604651163
--------------------
* Epoch 6/100


Loss: 0.69 #nan->0: 100%|██████████| 689/689 [17:28<00:00,  1.52s/it]


loss: 0.717880
acc: 0.507983
start validation


Loss: 0.67 #nan->0: 100%|██████████| 172/172 [01:34<00:00,  1.81it/s]


validation loss: 0.697965681552887
validation accuracy: 0.5174418604651163
--------------------


-----------------------

In [ ]:
freq=16
duration=6
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.82 #nan->0: 100%|██████████| 689/689 [19:03<00:00,  1.66s/it]


loss: 0.731373
acc: 0.515058
start validation


Loss: 0.65 #nan->0: 100%|██████████| 172/172 [01:38<00:00,  1.74it/s]


validation loss: 0.6822738647460938
validation accuracy: 0.5486918604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.6594850420951843
validation accuracy: 0.7049418604651163
--------------------
* Epoch 10/100


Loss: 0.31 #nan->0: 100%|██████████| 689/689 [17:54<00:00,  1.56s/it]


loss: 0.454198
acc: 0.798984
start validation


Loss: 0.64 #nan->0: 100%|██████████| 172/172 [01:37<00:00,  1.76it/s]

validation loss: 0.7065055966377258
validation accuracy: 0.6940406976744186
--------------------


-----------------------

In [ ]:
freq=16
duration=6
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =3e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.89 #nan->0: 100%|██████████| 689/689 [18:06<00:00,  1.58s/it]


loss: 0.775685
acc: 0.497642
start validation


Loss: 0.67 #nan->0: 100%|██████████| 172/172 [01:32<00:00,  1.85it/s]


validation loss: 0.6960111856460571
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.707823
acc: 0.505624
start validation


Loss: 0.69 #nan->0: 100%|██████████| 172/172 [01:32<00:00,  1.86it/s]


validation loss: 0.6928023099899292
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 6/100


Loss: 0.73 #nan->0:   9%|▉         | 61/689 [01:34<16:14,  1.55s/it]


KeyboardInterrupt: ignored

-----------------

In [ ]:
freq=4
duration=6
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =3e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.61 #nan->0: 100%|██████████| 689/689 [04:57<00:00,  2.32it/s]


loss: 0.718314
acc: 0.532293
start validation


Loss: 0.69 #nan->0: 100%|██████████| 172/172 [00:27<00:00,  6.29it/s]


validation loss: 0.6616636514663696
validation accuracy: 0.6119186046511628
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.6393231153488159
validation accuracy: 0.7361918604651163
--------------------
* Epoch 7/100


Loss: 0.18 #nan->0: 100%|██████████| 689/689 [04:57<00:00,  2.32it/s]


loss: 0.248777
acc: 0.903665
start validation


Loss: 0.43 #nan->0: 100%|██████████| 172/172 [00:27<00:00,  6.32it/s]


validation loss: 0.8362935185432434
validation accuracy: 0.7231104651162791
--------------------


-----------------------

In [ ]:
freq=4
duration=6
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.87 #nan->0: 100%|██████████| 689/689 [05:45<00:00,  1.99it/s]


loss: 0.723931
acc: 0.529028
start validation


Loss: 0.67 #nan->0: 100%|██████████| 172/172 [00:26<00:00,  6.52it/s]


validation loss: 0.6663545966148376
validation accuracy: 0.5806686046511628
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.7959058880805969
validation accuracy: 0.7100290697674418
--------------------
* Epoch 11/100


Loss: 0.18 #nan->0: 100%|██████████| 689/689 [04:51<00:00,  2.37it/s]


loss: 0.308771
acc: 0.871916
start validation


Loss: 0.86 #nan->0: 100%|██████████| 172/172 [00:26<00:00,  6.54it/s]


validation loss: 0.8153096437454224
validation accuracy: 0.717296511627907
--------------------


----------------

In [ ]:
freq=4
duration=6
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =2e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.66 #nan->0: 100%|██████████| 689/689 [04:51<00:00,  2.37it/s]


loss: 0.722683
acc: 0.539369
start validation


Loss: 0.67 #nan->0: 100%|██████████| 172/172 [00:26<00:00,  6.50it/s]


validation loss: 0.6455203294754028
validation accuracy: 0.6206395348837209
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.8035845160484314
validation accuracy: 0.7332848837209303
--------------------
* Epoch 9/100


Loss: 0.21 #nan->0: 100%|██████████| 689/689 [04:52<00:00,  2.36it/s]


loss: 0.202537
acc: 0.919993
start validation


Loss: 1.14 #nan->0: 100%|██████████| 172/172 [00:26<00:00,  6.54it/s]


validation loss: 0.8425167798995972
validation accuracy: 0.7456395348837209
--------------------


-------------------------

----------------------

In [ ]:
freq=16
duration=3
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 3secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.81 #nan->0: 100%|██████████| 689/689 [12:16<00:00,  1.07s/it]


loss: 0.765378
acc: 0.505987
start validation


Loss: 0.66 #nan->0: 100%|██████████| 172/172 [01:04<00:00,  2.67it/s]


validation loss: 0.7067013382911682
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.6930947303771973
validation accuracy: 0.5174418604651163
--------------------
* Epoch 8/100


Loss: 0.73 #nan->0: 100%|██████████| 689/689 [11:26<00:00,  1.00it/s]


loss: 0.716337
acc: 0.500000
start validation


Loss: 0.68 #nan->0: 100%|██████████| 172/172 [01:04<00:00,  2.65it/s]


validation loss: 0.6931987404823303
validation accuracy: 0.5174418604651163
--------------------


-----------------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 3secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.71 #nan->0: 100%|██████████| 689/689 [11:27<00:00,  1.00it/s]


loss: 0.720372
acc: 0.534652
start validation


Loss: 0.65 #nan->0: 100%|██████████| 172/172 [01:05<00:00,  2.63it/s]


validation loss: 0.6689531803131104
validation accuracy: 0.5864825581395349
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.38 #nan->0: 100%|██████████| 689/689 [11:26<00:00,  1.00it/s]


loss: 0.426614
acc: 0.806967
start validation


Loss: 0.42 #nan->0: 100%|██████████| 172/172 [01:05<00:00,  2.64it/s]


validation loss: 0.6267155408859253
validation accuracy: 0.7311046511627907
--------------------
* Epoch 10/100


Loss: 0.37 #nan->0:  42%|████▏     | 291/689 [04:52<06:35,  1.01it/s]

-----------------------

In [ ]:
freq=16
duration=3
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =3e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 3secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.76 #nan->0: 100%|██████████| 689/689 [12:50<00:00,  1.12s/it]


loss: 0.720902
acc: 0.539550
start validation


Loss: 0.65 #nan->0: 100%|██████████| 172/172 [01:06<00:00,  2.60it/s]


validation loss: 0.6600566506385803
validation accuracy: 0.6053779069767442
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.7323154807090759
validation accuracy: 0.7398255813953488
--------------------
* Epoch 9/100


Loss: 0.01 #nan->0: 100%|██████████| 689/689 [12:00<00:00,  1.05s/it]


loss: 0.209857
acc: 0.918360
start validation


Loss: 0.42 #nan->0: 100%|██████████| 172/172 [01:06<00:00,  2.59it/s]


validation loss: 0.8975287079811096
validation accuracy: 0.7543604651162791
--------------------


--------------------------

-------------------

In [ ]:
freq=4
duration=3
train_ds = AudiosDataset(train_binary['filename'], train_binary['label'])
test_ds = AudiosDataset(test_binary['filename'], test_binary['label'])
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =8e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 3secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.67 #nan->0: 100%|██████████| 689/689 [03:43<00:00,  3.08it/s]


loss: 0.765505
acc: 0.510885
start validation


Loss: 0.67 #nan->0: 100%|██████████| 172/172 [00:20<00:00,  8.55it/s]


validation loss: 0.7440200448036194
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.7157666087150574
validation accuracy: 0.48255813953488375
--------------------
* Epoch 8/100


Loss: 0.69 #nan->0: 100%|██████████| 689/689 [03:44<00:00,  3.07it/s]


loss: 0.726617
acc: 0.510522
start validation


Loss: 0.66 #nan->0: 100%|██████████| 172/172 [00:19<00:00,  8.69it/s]


validation loss: 0.7188506126403809
validation accuracy: 0.5174418604651163
--------------------


-----------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 3secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.74 #nan->0: 100%|██████████| 689/689 [03:44<00:00,  3.06it/s]


loss: 0.717280
acc: 0.530479
start validation


Loss: 0.65 #nan->0: 100%|██████████| 172/172 [00:20<00:00,  8.54it/s]


validation loss: 0.6628274917602539
validation accuracy: 0.5843023255813954
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.7132665514945984
validation accuracy: 0.6918604651162791
--------------------
* Epoch 10/100


Loss: 0.43 #nan->0: 100%|██████████| 689/689 [03:44<00:00,  3.07it/s]


loss: 0.322274
acc: 0.869920
start validation


Loss: 0.63 #nan->0: 100%|██████████| 172/172 [00:20<00:00,  8.57it/s]


validation loss: 0.787662148475647
validation accuracy: 0.7078488372093024
--------------------


-----------------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =3e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#4mhz 3secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.66 #nan->0: 100%|██████████| 689/689 [03:45<00:00,  3.06it/s]


loss: 0.724337
acc: 0.519231
start validation


Loss: 0.67 #nan->0: 100%|██████████| 172/172 [00:20<00:00,  8.22it/s]


validation loss: 0.6863992810249329
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.6209877133369446
validation accuracy: 0.7005813953488372
--------------------
* Epoch 16/100


Loss: 0.90 #nan->0: 100%|██████████| 689/689 [03:45<00:00,  3.06it/s]


loss: 0.513086
acc: 0.752540
start validation


Loss: 0.61 #nan->0: 100%|██████████| 172/172 [00:20<00:00,  8.44it/s]

validation loss: 0.630911111831665
validation accuracy: 0.6954941860465116
--------------------


--------------

----------------------

------------------------

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#16mhz 6secondi tanh dropout=0.7 batch=8 mockingjay

* Epoch 1/100


Loss: 0.70 #nan->0: 100%|██████████| 689/689 [18:29<00:00,  1.61s/it]


loss: 0.725511
acc: 0.533019
start validation


Loss: 0.69 #nan->0: 100%|██████████| 172/172 [01:34<00:00,  1.83it/s]


validation loss: 0.6829233169555664
validation accuracy: 0.5494186046511628
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.53 #nan->0: 100%|██████████| 689/689 [17:31<00:00,  1.53s/it]


loss: 0.578258
acc: 0.713534
start validation


Loss: 0.78 #nan->0: 100%|██████████| 172/172 [01:34<00:00,  1.83it/s]


validation loss: 0.6807129383087158
validation accuracy: 0.6504360465116279
--------------------
* Epoch 8/100


Loss: 0.35 #nan->0:  36%|███▌      | 249/689 [06:18<10:22,  1.41s/it]

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi tanh dropout=0.7 batch=12 mockingjay

* Epoch 1/100


Loss: 0.72 #nan->0: 100%|██████████| 459/459 [04:13<00:00,  1.81it/s]


loss: 0.742540
acc: 0.516158
start validation


Loss: 0.69 #nan->0: 100%|██████████| 115/115 [00:20<00:00,  5.49it/s]


validation loss: 0.6735999584197998
validation accuracy: 0.5869565217391305
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.24 #nan->0: 100%|██████████| 459/459 [03:19<00:00,  2.30it/s]


loss: 0.310148
acc: 0.875272
start validation


Loss: 0.34 #nan->0: 100%|██████████| 115/115 [00:21<00:00,  5.46it/s]


validation loss: 0.7296984195709229
validation accuracy: 0.7318840579710145
--------------------
* Epoch 12/100


Loss: 0.09 #nan->0:  73%|███████▎  | 333/459 [02:25<00:54,  2.32it/s]

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#16mhz 3secondi tanh dropout=0.7 batch=12 mockingjay

* Epoch 1/100


Loss: 0.58 #nan->0: 100%|██████████| 459/459 [11:48<00:00,  1.54s/it]


loss: 0.733843
acc: 0.532317
start validation


Loss: 0.67 #nan->0: 100%|██████████| 115/115 [01:03<00:00,  1.81it/s]


validation loss: 0.6564988493919373
validation accuracy: 0.613768115942029
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.144864
acc: 0.950436
start validation


Loss: 0.55 #nan->0: 100%|██████████| 115/115 [01:03<00:00,  1.81it/s]


validation loss: 0.8379644751548767
validation accuracy: 0.7659420289855072
--------------------
* Epoch 11/100


Loss: 0.08 #nan->0:   2%|▏         | 9/459 [00:17<14:59,  2.00s/it]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#16mhz 3secondi tanh dropout=0.7 batch=12 mockingjay

* Epoch 1/100


Loss: 0.78 #nan->0: 100%|██████████| 459/459 [12:22<00:00,  1.62s/it]


loss: 0.734723
acc: 0.511619
start validation


Loss: 0.69 #nan->0: 100%|██████████| 115/115 [01:04<00:00,  1.77it/s]


validation loss: 0.6902628540992737
validation accuracy: 0.5601449275362319
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.67 #nan->0: 100%|██████████| 459/459 [11:25<00:00,  1.49s/it]


loss: 0.713824
acc: 0.554829
start validation


Loss: 0.68 #nan->0: 100%|██████████| 115/115 [01:04<00:00,  1.79it/s]


validation loss: 0.6747456192970276
validation accuracy: 0.5710144927536231
Save new model!
--------------------
* Epoch 9/100


Loss: 0.60 #nan->0:  38%|███▊      | 173/459 [04:19<07:03,  1.48s/it]

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#16mhz 3secondi tanh dropout=0.7 batch=12 mockingjay

* Epoch 1/100


Loss: 0.64 #nan->0: 100%|██████████| 459/459 [12:17<00:00,  1.61s/it]


loss: 0.717937
acc: 0.548293
start validation


Loss: 0.68 #nan->0: 100%|██████████| 115/115 [01:04<00:00,  1.79it/s]


validation loss: 0.67502361536026
validation accuracy: 0.5739130434782609
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.399031
acc: 0.831518
start validation


Loss: 0.50 #nan->0: 100%|██████████| 115/115 [01:04<00:00,  1.78it/s]


validation loss: 0.681235134601593
validation accuracy: 0.6978260869565217
--------------------
* Epoch 12/100


Loss: 0.29 #nan->0:  45%|████▍     | 205/459 [05:07<06:21,  1.50s/it]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#16mhz 3secondi tanh dropout=0.7 batch=12 mockingjay

* Epoch 1/100


Loss: 0.81 #nan->0: 100%|██████████| 459/459 [10:43<00:00,  1.40s/it]


loss: 0.775436
acc: 0.503994
start validation


Loss: 0.70 #nan->0: 100%|██████████| 115/115 [01:02<00:00,  1.83it/s]


validation loss: 0.6939883232116699
validation accuracy: 0.4826086956521739
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.74 #nan->0: 100%|██████████| 459/459 [10:39<00:00,  1.39s/it]


loss: 0.733534
acc: 0.504176
start validation


Loss: 0.70 #nan->0: 100%|██████████| 115/115 [01:01<00:00,  1.87it/s]


validation loss: 0.6937925219535828
validation accuracy: 0.4826086956521739
--------------------
* Epoch 10/100


Loss: 0.71 #nan->0:  76%|███████▋  | 351/459 [08:10<02:29,  1.38s/it]

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#16mhz 3secondi tanh dropout=0.7 batch=12 mockingjay

* Epoch 1/100


Loss: 0.64 #nan->0: 100%|██████████| 459/459 [11:31<00:00,  1.51s/it]


loss: 0.759883
acc: 0.502723
start validation


Loss: 0.68 #nan->0: 100%|██████████| 115/115 [01:01<00:00,  1.86it/s]


validation loss: 0.6940878033638
validation accuracy: 0.5173913043478261
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.726432
acc: 0.519608
start validation


Loss: 0.69 #nan->0: 100%|██████████| 115/115 [01:01<00:00,  1.87it/s]


validation loss: 0.6926138401031494
validation accuracy: 0.5173913043478261
Save new model!
--------------------
* Epoch 5/100


Loss: 0.85 #nan->0:  17%|█▋        | 78/459 [01:53<09:13,  1.45s/it]


KeyboardInterrupt: ignored

#### audio albert


In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#16mhz 3secondi tanh dropout=0.7 batch=12

* Epoch 1/100


Loss: 0.67 #nan->0: 100%|██████████| 459/459 [03:53<00:00,  1.97it/s]


loss: 0.690515
acc: 0.555556
start validation


Loss: 0.66 #nan->0: 100%|██████████| 115/115 [00:36<00:00,  3.15it/s]


validation loss: 0.6301531195640564
validation accuracy: 0.6536231884057971
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.200593
acc: 0.921932
start validation


Loss: 1.43 #nan->0: 100%|██████████| 115/115 [00:37<00:00,  3.08it/s]


validation loss: 0.7797514796257019
validation accuracy: 0.7340579710144928
--------------------
* Epoch 9/100


Loss: 0.05 #nan->0:  61%|██████    | 280/459 [01:54<01:13,  2.45it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi tanh dropout=0.7 batch=12

* Epoch 1/100


Loss: 0.92 #nan->0: 100%|██████████| 459/459 [01:52<00:00,  4.09it/s]


loss: 0.686976
acc: 0.555919
start validation


Loss: 0.73 #nan->0: 100%|██████████| 115/115 [00:13<00:00,  8.31it/s]


validation loss: 0.6566121578216553
validation accuracy: 0.6028985507246377
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.022828
acc: 0.993101
start validation


Loss: 1.87 #nan->0: 100%|██████████| 115/115 [00:13<00:00,  8.34it/s]


validation loss: 1.5146046876907349
validation accuracy: 0.7268115942028985
--------------------
* Epoch 41/100


Loss: 0.00 #nan->0:  12%|█▏        | 54/459 [00:09<01:09,  5.84it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.7

* Epoch 1/100


Loss: 0.66 #nan->0: 100%|██████████| 689/689 [02:01<00:00,  5.69it/s]


loss: 0.682495
acc: 0.562409
start validation


Loss: 0.64 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.81it/s]


validation loss: 0.6448449492454529
validation accuracy: 0.6329941860465116
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 1.8822665214538574
validation accuracy: 0.7143895348837209
--------------------
* Epoch 31/100


Loss: 0.00 #nan->0: 100%|██████████| 689/689 [01:10<00:00,  9.83it/s]


loss: 0.032987
acc: 0.989478
start validation


Loss: 0.30 #nan->0:  25%|██▌       | 43/172 [00:05<00:15,  8.39it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 6secondi relu dropout=0.7

* Epoch 1/100


Loss: 0.34 #nan->0: 100%|██████████| 689/689 [02:12<00:00,  5.21it/s]


loss: 0.686376
acc: 0.561865
start validation


Loss: 0.60 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.78it/s]


validation loss: 0.63966965675354
validation accuracy: 0.6329941860465116
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.109003
acc: 0.960631
start validation


Loss: 0.28 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.13it/s]


validation loss: 1.0214635133743286
validation accuracy: 0.7434593023255814
--------------------
* Epoch 11/100


  0%|          | 0/689 [00:00<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#5mhz 6secondi relu dropout=0.5

* Epoch 1/100


Loss: 0.59 #nan->0: 100%|██████████| 689/689 [02:26<00:00,  4.71it/s]


loss: 0.674037
acc: 0.577286
start validation


Loss: 0.63 #nan->0: 100%|██████████| 172/172 [00:17<00:00,  9.86it/s]


validation loss: 0.6237277388572693
validation accuracy: 0.6686046511627907
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.191441
acc: 0.925435
start validation


Loss: 0.45 #nan->0: 100%|██████████| 172/172 [00:17<00:00,  9.69it/s]


validation loss: 0.752937376499176
validation accuracy: 0.7238372093023255
--------------------
* Epoch 8/100


Loss: 0.09 #nan->0:  14%|█▍        | 95/689 [00:14<01:33,  6.34it/s]


KeyboardInterrupt: ignored

In [ ]:
#quello prima dalla 4° epoca (74%, 71%) ma con lr=1e-5
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/AudioAlbert/OK-healthy-unhealthy_aalbert_1e-4_relu_6sec_5mhz.bin'))
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#5mhz 6secondi relu dropout=0.5 lr=1e-5

* Epoch 1/100


Loss: 0.32 #nan->0: 100%|██████████| 689/689 [01:34<00:00,  7.32it/s]


loss: 0.378235
acc: 0.831640
start validation


Loss: 0.54 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.22it/s]


validation loss: 0.6146918535232544
validation accuracy: 0.7311046511627907
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.132549
acc: 0.953193
start validation


Loss: 0.59 #nan->0: 100%|██████████| 172/172 [00:17<00:00, 10.04it/s]


validation loss: 0.9501494765281677
validation accuracy: 0.7194767441860465
--------------------
* Epoch 13/100


Loss: 0.05 #nan->0:  94%|█████████▍| 651/689 [01:29<00:05,  7.24it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.5

* Epoch 1/100


Loss: 0.69 #nan->0: 100%|██████████| 689/689 [01:11<00:00,  9.69it/s]


loss: 0.686886
acc: 0.546081
start validation


Loss: 0.64 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.63it/s]


validation loss: 0.6617732048034668
validation accuracy: 0.5901162790697675
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.205552
acc: 0.916364
start validation


Loss: 0.18 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 10.95it/s]


validation loss: 0.8970562219619751
validation accuracy: 0.7231104651162791
--------------------
* Epoch 24/100


Loss: 0.28 #nan->0:   6%|▌         | 43/689 [00:06<01:38,  6.55it/s]


KeyboardInterrupt: ignored

In [ ]:
#quello prima dalla 7° epoca (71%, 69%) ma con lr=1e-6
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/AudioAlbert/OK-healthy-unhealthy_aalbert_1e-5_relu_dropout=0.5.bin'))
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.5 lr=1e-6

* Epoch 1/100


Loss: 0.93 #nan->0: 100%|██████████| 689/689 [01:10<00:00,  9.73it/s]


loss: 0.525552
acc: 0.744376
start validation


Loss: 0.42 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.44it/s]


validation loss: 0.5810795426368713
validation accuracy: 0.6984011627906976
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.486676
acc: 0.769049
start validation


Loss: 0.40 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.51it/s]


validation loss: 0.5899998545646667
validation accuracy: 0.6991279069767442
--------------------
* Epoch 11/100


Loss: 0.56 #nan->0:  32%|███▏      | 218/689 [00:24<00:52,  9.00it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi tanh dropout=0.7

* Epoch 1/100


Loss: 0.54 #nan->0: 100%|██████████| 689/689 [01:08<00:00, 10.00it/s]


loss: 0.694975
acc: 0.565312
start validation


Loss: 0.63 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 12.18it/s]


validation loss: 0.6503216624259949
validation accuracy: 0.6002906976744186
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.300177
acc: 0.872823
start validation


Loss: 0.90 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.88it/s]


validation loss: 0.7846303582191467
validation accuracy: 0.7056686046511628
--------------------
* Epoch 7/100


Loss: 0.16 #nan->0:   6%|▌         | 38/689 [00:05<01:38,  6.60it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.7

* Epoch 1/100


Loss: 0.86 #nan->0: 100%|██████████| 689/689 [01:59<00:00,  5.76it/s]


loss: 0.682115
acc: 0.560232
start validation


Loss: 0.77 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.64it/s]


validation loss: 0.6899991035461426
validation accuracy: 0.5472383720930233
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.179233
acc: 0.930697
start validation


Loss: 1.25 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.83it/s]


validation loss: 0.8176172971725464
validation accuracy: 0.7049418604651163
--------------------
* Epoch 9/100


Loss: 0.18 #nan->0:  62%|██████▏   | 430/689 [00:44<00:26,  9.67it/s]


KeyboardInterrupt: ignored

In [ ]:
#quello prima dalla 7°5° epoca (76%, 71%) ma con lr=1e-5
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/AudioAlbert/OK-healthy-unhealthy_aalbert_1e-4_relu.bin'))
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.5 lr=1e-5

* Epoch 1/100


Loss: 0.70 #nan->0: 100%|██████████| 689/689 [01:10<00:00,  9.82it/s]


loss: 0.339140
acc: 0.855225
start validation


Loss: 0.55 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.72it/s]


validation loss: 0.68845134973526
validation accuracy: 0.717296511627907
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.234502
acc: 0.905479
start validation


Loss: 0.82 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.56it/s]


validation loss: 0.8110164999961853
validation accuracy: 0.7151162790697675
--------------------
* Epoch 5/100


Loss: 0.37 #nan->0:  11%|█         | 73/689 [00:09<01:20,  7.63it/s]


KeyboardInterrupt: ignored

#### training con dataset sbagliato

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.7

* Epoch 1/100


Loss: 0.55 #nan->0: 100%|██████████| 689/689 [02:01<00:00,  5.67it/s]


loss: 0.573213
acc: 0.724601
start validation


Loss: 0.46 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 12.12it/s]


validation loss: 0.5274064540863037
validation accuracy: 0.7180232558139535
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.232231
acc: 0.900943
start validation


Loss: 1.31 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.73it/s]


validation loss: 0.6630783677101135
validation accuracy: 0.7739825581395349
--------------------
* Epoch 10/100


  0%|          | 0/689 [00:00<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu dropout=0.7

* Epoch 1/100


Loss: 0.67 #nan->0: 100%|██████████| 689/689 [02:00<00:00,  5.73it/s]


loss: 0.691341
acc: 0.573113
start validation


Loss: 0.57 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.89it/s]


validation loss: 0.623408317565918
validation accuracy: 0.7180232558139535
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.573078
acc: 0.732946
start validation


Loss: 0.44 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.81it/s]


validation loss: 0.5683332681655884
validation accuracy: 0.7180232558139535
Save new model!
--------------------
* Epoch 10/100


Loss: 0.60 #nan->0:  91%|█████████ | 628/689 [01:03<00:06,  9.97it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz 3secondi relu deep dropout=0.7 e 0.3

* Epoch 1/100


Loss: 0.38 #nan->0: 100%|██████████| 689/689 [02:02<00:00,  5.63it/s]


loss: 0.565083
acc: 0.730769
start validation


Loss: 0.45 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.78it/s]


validation loss: 0.5076175928115845
validation accuracy: 0.7558139534883721
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.047852
acc: 0.982221
start validation


Loss: 0.63 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.63it/s]


validation loss: 1.0516650676727295
validation accuracy: 0.7819767441860465
--------------------
* Epoch 14/100


Loss: 0.07 #nan->0:   2%|▏         | 14/689 [00:03<02:40,  4.19it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.7 3secondi dataset relu

* Epoch 1/100


Loss: 1.08 #nan->0: 100%|██████████| 689/689 [01:58<00:00,  5.82it/s]


loss: 0.572104
acc: 0.720791
start validation


Loss: 0.43 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.88it/s]


validation loss: 0.5387810468673706
validation accuracy: 0.7180232558139535
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.019588
acc: 0.994376
start validation


Loss: 0.11 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.52it/s]


validation loss: 1.33577561378479
validation accuracy: 0.7914244186046512
--------------------
* Epoch 65/100


Loss: 0.08 #nan->0:   5%|▍         | 34/689 [00:05<01:45,  6.18it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.7 3secondi dataset balanced

* Epoch 1/100


Loss: 0.74 #nan->0: 100%|██████████| 387/387 [00:40<00:00,  9.65it/s]


loss: 0.710685
acc: 0.533592
start validation


Loss: 0.69 #nan->0: 100%|██████████| 97/97 [00:08<00:00, 11.15it/s]


validation loss: 0.662943959236145
validation accuracy: 0.6301546391752577
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.373187
acc: 0.836240
start validation


Loss: 0.14 #nan->0: 100%|██████████| 97/97 [00:08<00:00, 11.16it/s]


validation loss: 0.7514455914497375
validation accuracy: 0.720360824742268
--------------------
* Epoch 18/100


Loss: 0.42 #nan->0:   9%|▊         | 33/387 [00:05<00:55,  6.35it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.5 3secondi classificatore deep

* Epoch 1/100


Loss: 0.59 #nan->0: 100%|██████████| 689/689 [02:03<00:00,  5.57it/s]


loss: 0.546557
acc: 0.744557
start validation


Loss: 0.43 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.50it/s]


validation loss: 0.5090526938438416
validation accuracy: 0.7638081395348837
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.054980
acc: 0.979318
start validation


Loss: 0.87 #nan->0: 100%|██████████| 172/172 [00:14<00:00, 11.70it/s]


validation loss: 0.9800843596458435
validation accuracy: 0.7761627906976745
--------------------
* Epoch 11/100


Loss: 0.31 #nan->0:  25%|██▍       | 169/689 [00:19<00:58,  8.88it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.5 3secondi split=0.4

* Epoch 1/100


Loss: 0.66 #nan->0: 100%|██████████| 517/517 [01:40<00:00,  5.16it/s]


loss: 0.564747
acc: 0.732592
start validation


Loss: 0.32 #nan->0: 100%|██████████| 345/345 [00:23<00:00, 14.66it/s]


validation loss: 0.5135382413864136
validation accuracy: 0.7692028985507247
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.00 #nan->0: 100%|██████████| 517/517 [00:45<00:00, 11.35it/s]


loss: 0.021908
acc: 0.992747
start validation


Loss: 0.04 #nan->0: 100%|██████████| 345/345 [00:23<00:00, 14.62it/s]


validation loss: 1.2161269187927246
validation accuracy: 0.7927536231884058
--------------------
* Epoch 59/100


Loss: 0.00 #nan->0:  77%|███████▋  | 396/517 [00:35<00:10, 11.86it/s]

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.7 6secondi

* Epoch 1/100


Loss: 0.87 #nan->0: 100%|██████████| 689/689 [01:16<00:00,  9.04it/s]


loss: 0.593195
acc: 0.711901
start validation


Loss: 0.43 #nan->0: 100%|██████████| 172/172 [00:12<00:00, 13.91it/s]


validation loss: 0.5598057508468628
validation accuracy: 0.7180232558139535
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.291185
acc: 0.876996
start validation


Loss: 0.82 #nan->0: 100%|██████████| 172/172 [00:12<00:00, 14.26it/s]


validation loss: 0.5884414911270142
validation accuracy: 0.7659883720930233
--------------------
* Epoch 16/100


Loss: 0.08 #nan->0:  32%|███▏      | 220/689 [00:25<00:54,  8.63it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.7 6secondi

* Epoch 1/100


Loss: 0.47 #nan->0: 100%|██████████| 689/689 [01:15<00:00,  9.11it/s]


loss: 0.560946
acc: 0.730588
start validation


Loss: 0.44 #nan->0: 100%|██████████| 172/172 [00:12<00:00, 13.68it/s]


validation loss: 0.5140737295150757
validation accuracy: 0.7521802325581395
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.210140
acc: 0.914731
start validation


Loss: 0.57 #nan->0: 100%|██████████| 172/172 [00:12<00:00, 13.61it/s]


validation loss: 0.6174583435058594
validation accuracy: 0.7659883720930233
--------------------
* Epoch 7/100


Loss: 0.06 #nan->0:  14%|█▍        | 98/689 [00:12<01:13,  8.04it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz dropout=0.5

* Epoch 1/100


Loss: 0.58 #nan->0: 100%|██████████| 689/689 [02:03<00:00,  5.60it/s]


loss: 0.554238
acc: 0.731858
start validation


Loss: 0.46 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.69it/s]


validation loss: 0.5173416137695312
validation accuracy: 0.7390988372093024
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.022887
acc: 0.990747
start validation


Loss: 1.72 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.10it/s]


validation loss: 1.19481360912323
validation accuracy: 0.7725290697674418
--------------------
* Epoch 35/100


Loss: 0.00 #nan->0:  62%|██████▏   | 427/689 [00:46<00:28,  9.09it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz EXTENDED dropout=0.5

* Epoch 1/100


Loss: 0.34 #nan->0: 100%|██████████| 689/689 [02:30<00:00,  4.57it/s]


loss: 0.549004
acc: 0.742017
start validation


Loss: 0.44 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.59it/s]


validation loss: 0.5087576508522034
validation accuracy: 0.7521802325581395
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.191890
acc: 0.921626
start validation


Loss: 0.56 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.60it/s]


validation loss: 0.6840699911117554
validation accuracy: 0.7805232558139535
--------------------
* Epoch 6/100


Loss: 0.55 #nan->0:  15%|█▌        | 104/689 [00:23<02:13,  4.38it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz EXTENDED

* Epoch 1/100


Loss: 0.46 #nan->0: 100%|██████████| 689/689 [03:24<00:00,  3.37it/s]


loss: 0.570584
acc: 0.725871
start validation


Loss: 0.44 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.61it/s]


validation loss: 0.5190539956092834
validation accuracy: 0.7492732558139535
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.046427
acc: 0.984216
start validation


Loss: 1.44 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.58it/s]


validation loss: 0.9704733490943909
validation accuracy: 0.7936046511627907
--------------------
* Epoch 12/100


Loss: 0.15 #nan->0:  98%|█████████▊| 677/689 [02:28<00:02,  4.56it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#4mhz

* Epoch 1/100


Loss: 0.62 #nan->0: 100%|██████████| 689/689 [01:11<00:00,  9.63it/s]


loss: 0.573839
acc: 0.726415
start validation


Loss: 0.48 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.44it/s]


validation loss: 0.5224190354347229
validation accuracy: 0.7507267441860465
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.064792
acc: 0.976597
start validation


Loss: 0.46 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.24it/s]


validation loss: 1.109357476234436
validation accuracy: 0.7638081395348837
--------------------
* Epoch 12/100


Loss: 0.01 #nan->0:  91%|█████████ | 626/689 [01:07<00:06,  9.26it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#7mhz

* Epoch 1/100


Loss: 0.30 #nan->0: 100%|██████████| 689/689 [01:26<00:00,  7.93it/s]


loss: 0.559725
acc: 0.724964
start validation


Loss: 0.33 #nan->0: 100%|██████████| 172/172 [00:11<00:00, 15.22it/s]


validation loss: 0.497946172952652
validation accuracy: 0.7623546511627907
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.109533
acc: 0.960450
start validation


Loss: 0.67 #nan->0: 100%|██████████| 172/172 [00:11<00:00, 15.29it/s]


validation loss: 0.7650411128997803
validation accuracy: 0.7936046511627907
--------------------
* Epoch 9/100


Loss: 0.01 #nan->0:   4%|▍         | 28/689 [00:05<01:59,  5.55it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#5mhz

* Epoch 1/100


Loss: 0.76 #nan->0: 100%|██████████| 689/689 [01:22<00:00,  8.38it/s]


loss: 0.572365
acc: 0.724057
start validation


Loss: 0.52 #nan->0: 100%|██████████| 172/172 [00:16<00:00, 10.20it/s]


validation loss: 0.5210419297218323
validation accuracy: 0.7420058139534884
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.099157
acc: 0.962083
start validation


Loss: 1.81 #nan->0: 100%|██████████| 172/172 [00:17<00:00, 10.06it/s]


validation loss: 0.7372713088989258
validation accuracy: 0.8081395348837209
--------------------
* Epoch 10/100


Loss: 0.04 #nan->0:  11%|█         | 74/689 [00:10<01:31,  6.74it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#2mhz

* Epoch 1/100


Loss: 0.44 #nan->0: 100%|██████████| 689/689 [00:55<00:00, 12.52it/s]


loss: 0.577725
acc: 0.726778
start validation


Loss: 0.52 #nan->0: 100%|██████████| 172/172 [00:11<00:00, 15.24it/s]


validation loss: 0.5585514307022095
validation accuracy: 0.7325581395348837
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.050927
acc: 0.980951
start validation


Loss: 0.94 #nan->0: 100%|██████████| 172/172 [00:11<00:00, 14.95it/s]


validation loss: 1.3156260251998901
validation accuracy: 0.7463662790697675
--------------------
* Epoch 16/100


Loss: 0.01 #nan->0:   5%|▌         | 35/689 [00:04<01:27,  7.51it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#3mhz

* Epoch 1/100


Loss: 0.85 #nan->0: 100%|██████████| 689/689 [01:45<00:00,  6.53it/s]


loss: 0.588611
acc: 0.723331
start validation


Loss: 0.46 #nan->0: 100%|██████████| 172/172 [00:09<00:00, 17.41it/s]


validation loss: 0.5512516498565674
validation accuracy: 0.7238372093023255
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.180153
acc: 0.932511
start validation


Loss: 1.28 #nan->0: 100%|██████████| 172/172 [00:09<00:00, 18.14it/s]


validation loss: 0.8466366529464722
validation accuracy: 0.7572674418604651
--------------------
* Epoch 8/100


Loss: 0.04 #nan->0:  27%|██▋       | 188/689 [00:15<00:42, 11.92it/s]


KeyboardInterrupt: ignored

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])

* Epoch 1/100


Loss: 0.60 #nan->0: 100%|██████████| 689/689 [00:53<00:00, 12.91it/s]


loss: 0.604355
acc: 0.714623
start validation


Loss: 0.43 #nan->0: 100%|██████████| 172/172 [00:09<00:00, 17.87it/s]


validation loss: 0.572152853012085
validation accuracy: 0.7180232558139535
Save new model!
--------------------
* Epoch 2/100


...

loss: 0.088003
acc: 0.968070
start validation


Loss: 1.85 #nan->0: 100%|██████████| 172/172 [00:09<00:00, 17.21it/s]


validation loss: 1.095919132232666
validation accuracy: 0.7616279069767442
--------------------
* Epoch 37/100


Loss: 0.00 #nan->0:  84%|████████▍ | 581/689 [00:46<00:08, 12.61it/s]


KeyboardInterrupt: ignored